# full_automated_calibration
1. imports
2. robot initialisation and checks
3. finding flow_rate_aspirate
3. loop pipetting using 5 point initialisation parameters


### Imports

In [ ]:
#General Imports
from datetime import datetime, date
import os
import time
import pandas as pd
import numpy as np
import scipy
from scipy import signal
import re
from sklearn.linear_model import LinearRegression

# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title



### Robot initialisation and checks

In [ ]:
#Import robot related packages and run setup
import pandas as pd
import time
from matplotlib import pyplot as plt
from pathlib import Path
import sys
REPOS = 'GitHub'
ROOT = str(Path().absolute()).split(REPOS)[0]
sys.path.append(f'{ROOT}{REPOS}')

from polylectric.configs.SynthesisB1 import SETUP, LAYOUT_FILE

from controllably import load_deck      # optional
load_deck(SETUP.setup, LAYOUT_FILE)     # optional

platform = SETUP
platform.mover.verbose = False #askpablo

In [ ]:
#Initialization of variables for platform objects
pipette= platform.setup
deck = platform.setup.deck
balance = platform.balance
balance_deck = deck.slots['1']
source = deck.slots['2']
tip_rack = deck.slots['3']
bin = deck.slots['4']
pipette.mover.setSpeed(50)
print(balance_deck)
print(source)
print(tip_rack)
print(bin)

In [ ]:
#Check if balance is connected
balance.zero() #to tare
balance.toggleRecord(True) # turn on and record weight
time.sleep(5) # do previous action for 5s
print(balance.buffer_df.iloc[-1]) #iloc can take -1, loc needs to be 839 loc[839,["Time","Value","Factor","Baseline","Mass"]]. -1 is last line. to find number of last line, print(balance.buffer_df)
balance.toggleRecord(False) #turn off

In [ ]:
#Liquid information
liquid_name = "Viscosity_std_505"
density = 0.8683


In [ ]:
#Establish initial height of liquid on the source vial
pipette_name = 'rLine1000'
initial_liquid_level = 10 # in mm

### Finding flow_rate_aspirate

In [ ]:
#%%This cell can be used to measure the mass change profiles when you aspirate a liquid at different speed rates.
#The run is automatically stopped when the mass change derivative is close to 0
#folder to save files in/get files from
REPO = 'viscosity_liquid_transfer_Pablo'
folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments\Mass_balance_flow_Rate\Calibration'

#getting datetime info to save file as todays date
today = date.today()
today = today.strftime("%Y-%m-%d")
now = datetime.now(tz=None)
now = now.strftime("%H-%M-%S")
if  not os.path.exists(folder+'\\'+today):
    os.mkdir(folder+'\\'+today)
folder = folder+'\\'+today

# Setting starting parameters
speed =  265
volume=1000
liquid_name = 'Viscosity_std_505'
filename = folder + '/' +'/'+ today + "_" + now[:-3] + '_' +liquid_name+'_'+str(speed).replace('.','_') 
pipette.mover.setSpeed(50)
pipette.mover.setHandedness(False)
liquid_level = initial_liquid_level

#attach pipette tip
if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

#Moving to balance to aspirate
# if pipette.mover.getToolPosition()[0] != balance_deck.wells['A1'].from_top((0,0,-10)):
#     
pipette.mover.safeMoveTo(balance_deck.wells['A1'].from_bottom((0,0,liquid_level-5)),descent_speed_fraction=0.25)

#Starting balance measurement
time.sleep(5)
balance.zero(wait=5)
balance.clearCache()
balance.toggleRecord(on=True)
time.sleep(15)

pipette.liquid.aspirate(volume, speed=speed)

#Switching the balance off after change in mass is less than 0.05
while True:
    data = balance.buffer_df
    data['Mass_smooth']= signal.savgol_filter(data['Mass'],91,1)
    data['Mass_derivative_smooth']=data['Mass_smooth'].diff()
    condition=data['Mass_derivative_smooth'].rolling(30).mean().iloc[-1]
    if condition>-0.05:
        break
print('loop stopped')

#dispensing liquid at the rate specified
time.sleep(10)
pipette.mover.setSpeed(50)
pipette.mover.moveTo(balance_deck.wells['A1'].from_top((0,0,10)))
pipette.liquid.dispense(1000, speed=20)
time.sleep(10)
#switching balance off and saving csv
balance.toggleRecord(on=False)
balance.buffer_df.to_csv(filename + '.csv')

In [ ]:
#using data from balance buffer_df above, calculate time in seconds and mass derivatives
data['ts'] = data['Time'].astype('datetime64[ns]').values.astype('float') / 10 ** 9
data['ts']= data['ts']-data['ts'][0]
# data['Mass']=files_dict[key]['Mass']-files_dict[key]['Mass'][0]
data['Mass_analysis_smooth']= signal.savgol_filter(data['Mass'],91,89)
data['Mass_analysis_derivative_smooth']=data['Mass_analysis_smooth'].diff()
filename = folder + '/' +'/'+ today + "_" + now[:-3] + '_' +liquid_name+'_'+str(speed).replace('.','_') 
data.to_csv(filename + '_complete_data.csv')

In [ ]:
#Defining sigmoid function (to smooth out jagged curve and get a better derivative)
from scipy.optimize import curve_fit
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

#plotting time 
xdata = data.where(data['ts']>15).dropna()['ts']
ydata = data.where(data['ts']>15).dropna()['Mass']

p0 = [max(ydata)+30, np.median(xdata),1,min(ydata)] # this is an mandatory initial guess
print(p0)

popt, pcov = curve_fit(sigmoid, xdata, ydata,p0)

yfit = sigmoid(xdata,popt[0],popt[1],popt[2],popt[3])

    

fig,axs = plt.subplots(2)
axs[0].plot(xdata,ydata,color = 'red', label= liquid_name + ' raw data')
axs[0].plot(xdata,yfit,color = 'blue', label= 'sigmoid fit')
axs[0].legend()
axs[1].plot(xdata,yfit.diff(),color = 'green', label= 'fit derivative')
axs[1].plot(xdata,yfit.diff().rolling(4).mean(),color='red')
axs[1].legend()

In [ ]:
xdata = data.where(data['ts']>15).dropna()['ts']
ydata = data.where(data['ts']>15).dropna()['Mass']

p0 = [max(ydata)+30, np.median(xdata),1,min(ydata)] # this is an mandatory initial guess
print(p0)

popt, pcov = curve_fit(sigmoid, xdata, ydata,p0)

yfit = sigmoid(xdata,popt[0],popt[1],popt[2],popt[3])

diff_curve = yfit.diff().rolling(4).mean()

#get the index of the minimum differentiated mass/time function
min_index = np.where(diff_curve == diff_curve.min())[0][0]
#using min index to get average of dM/dt at min
initial_flow_rate_aspirate = - (diff_curve[min_index-5:min_index+5].mean() / density)



### Loop pipetting using 5 point initialisation parameters

In [ ]:
initial_flow_rate_aspirate 

In [ ]:
#New dataframe
df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out', 'delay_aspirate', 'delay_dispense', 'delay_blow_out', 'density', 'time', 'm', '%error', 'Transfer_Observation', 'Comment'])
df = df.astype({'liquid':str,'pipette':str,'blow_out':bool,'Transfer_Observation':str,'Comment':str})

#load dataframe if experiment was done halfway previously
# df = pd.read_csv(liquid_name+'.csv')

In [ ]:
liquids_dict = {
  liquid_name :{ # why so many dict #askpablo
        "rLine1000": {
            "aspiration_rate": initial_flow_rate_aspirate,
            "dispense_rate": initial_flow_rate_aspirate,
            "blow_out" : False,
            "delay_aspirate" : 10, 
            "delay_dispense" : 10, 
            "delay_blow_out" : 0, 
            },
    }

}

In [ ]:
#Check if new tip is required
pipette.mover.setSpeed(50)
pipette.mover.setHandedness(False)

if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

#setup for loops
#TO BE CHANGED (sometimes)
iterations = 5
volumes_list = [1000, 500, 300]

#NOT TO BE CHANGED
liquid_level = initial_liquid_level
counter = 5

#while loop
while counter <= iterations:

    #hardcoding aspirate and dispense rates:
    if counter == 1:
        liquids_dict[liquid_name][pipette_name]['aspiration_rate'] = initial_flow_rate_aspirate
        liquids_dict[liquid_name][pipette_name]['dispense_rate'] = initial_flow_rate_aspirate
    if counter == 2:
        liquids_dict[liquid_name][pipette_name]['aspiration_rate'] = 1.25 * initial_flow_rate_aspirate
        liquids_dict[liquid_name][pipette_name]['dispense_rate'] = 1.25 * initial_flow_rate_aspirate
    if counter == 3:
        liquids_dict[liquid_name][pipette_name]['aspiration_rate'] = 1.25 * initial_flow_rate_aspirate
        liquids_dict[liquid_name][pipette_name]['dispense_rate'] = 0.1 * initial_flow_rate_aspirate
    if counter == 4:
        liquids_dict[liquid_name][pipette_name]['aspiration_rate'] = 0.1 * initial_flow_rate_aspirate
        liquids_dict[liquid_name][pipette_name]['dispense_rate'] = 1.25 * initial_flow_rate_aspirate
    if counter == 5:
        liquids_dict[liquid_name][pipette_name]['aspiration_rate'] = 0.1 * initial_flow_rate_aspirate
        liquids_dict[liquid_name][pipette_name]['dispense_rate'] = 0.1 * initial_flow_rate_aspirate


    #for loop
    for volume in volumes_list:
        #liquid transfer
        #transfer start
        start = time.time() 

        #aspirate step
        pipette.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5))) 
        pipette.liquid.aspirate(volume, speed=liquids_dict[liquid_name][pipette_name]['aspiration_rate'])
        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_aspirate'])

        pipette.touchTip(source.wells['A1']) 

        #dispense step
        pipette.mover.safeMoveTo(balance_deck.wells['A1'].from_top((0,0,-5))) 
        balance.tare() 
        balance.clearCache() 
        balance.toggleRecord(True) 
        time.sleep(5)
        pipette.liquid.dispense(volume, speed=liquids_dict[liquid_name][pipette_name]['dispense_rate'])
        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_dispense'])

        #blowout step
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True: 
            pipette.liquid.blowout(home=False)
            time.sleep(liquids_dict[liquid_name][pipette_name]['delay_blow_out']) 

        #transfer termination
        finish = time.time() 
        time_m = finish - start

        pipette.mover.safeMoveTo(source.wells['A1'].top) 
        time.sleep(5)
        balance.toggleRecord(False) 
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
            pipette.liquid.home() 

        #do blowout
        pipette.liquid.blowout(home=False) 
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()

        #record transfer values 
        #calculating mass error functions
        m = (balance.buffer_df.iloc[-10:,-1].mean()-balance.buffer_df.iloc[:10,-1].mean())/1000 
        error = (m-density*volume/1000)/(density/1000*volume)*100

        #making new dataframe + filling it in
        df = pd.concat([df,pd.DataFrame(liquids_dict[liquid_name][pipette_name],index=[0])],ignore_index=True)
        df.iloc[-1,-4] = time_m
        df.iloc[-1,2] = volume
        df.iloc[-1, 0] = liquid_name
        df.iloc[-1, 1] = pipette_name
        df.iloc[-1,-5] = density
        df.iloc[-1, -3] = m
        df.iloc[-1,-2]= error

        #change liquid levels
        liquid_level = liquid_level - 1.2*m/density   

        #printing checks
        print("LIQUID LEVEL: " +str(liquid_level) + "   LIQUID CHANGE: " +str(1.2*m/density) + "   ITERATION: " + str(counter) + ", " + "VOLUME: " + str(volume))    

        #liquid level checks
        if (1.2*m/density > 1.2) or (1.2*m/density < 0):
            break
        if (liquid_level > initial_liquid_level) or (liquid_level < 6):
            break
     #liquid level checks
    if (1.2*m/density > 1.2) or (1.2*m/density < 0):
        break
    if (liquid_level > initial_liquid_level) or (liquid_level < 6):
        break
    
    counter += 1



In [ ]:
#To save the csv
df.to_csv(liquid_name+'.csv', index = False)

In [ ]:
pipette.mover.home()

In [ ]:
df = df.drop(7)

In [ ]:
balance.connect()

In [ ]:
fig,axs = plt.subplots()

axs.plot(balance.buffer_df['Time'],balance.buffer_df['Mass'])

axs.set_xlabel('Time')
axs.set_ylabel('Mass')

plt.show()

In [ ]:
pipette.ejectTipAt(bin.wells['A1'].top)
pipette.mover.home()